<a href="https://colab.research.google.com/github/natalykur/rag_tutorial/blob/main/RAG_inline_iphone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 Environment Setup and Installation
First, we need to install the necessary libraries. We'll use LangChain for handling the RAG pipeline components, LangChain Community for document loaders, a vector store (e.g., ChromaDB), and an embedding model.

In [ ]:
# Install required libraries
!pip install  sentence-transformers
!pip install -U langchain-community
!pip install openai pymupdf faiss-cpu scikit-learn


!pip install langchain
!pip install langchain-openai # If using OpenAI models for LLM and embeddings
!pip install chromadb
!pip install pypdf # For loading PDF files
!pip install ragas # For evaluation
!pip install langsmith # For tracking and tracing
!pip install unstructured # For advanced PDF parsing (optional)
!pip install --upgrade --quiet langgraph # For creating complex RAG workflows later

^C


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import files
import fitz  # PyMuPDF2

# The Problem data and model assumptions and definitions

in the following section I will prepare the system and problem definitition : load iphone instruction files and add evaluation set.
I'll also add my configuration for llm: the model gpt 3.5 and the hyperparameters

In [ ]:
4from google.colab import files
import fitz  # PyMuPDF4

## Set a way to call openapi

In [ ]:
from openai import OpenAI
from google.colab import userdata, files
import fitz  # PyMuPDF49

# Set your OpenAI API key
personal_api_key  = userdata.get("OPENAI_API_KEY")
  # Replace with your own API key
llm_model = "gpt-3.5-turbo"

client = OpenAI(
    # This is the default and can be omitted
    api_key=personal_api_key,
)

In [ ]:

temperature = 0.7  # Controls the randomness of the output. Higher values (closer to 1) result in more random outputs, while lower values (closer to 0) make the output more deterministic. A value of 0.7 is a common balance between creativity and predictability.
max_tokens = 500  # The maximum number of tokens (words, subwords, etc.) the model is allowed to generate in its response. Setting a limit helps control the length of the output and can prevent excessively long responses.
top_p = 1.0  # Controls the diversity of the output. It's a sampling strategy where the model only considers tokens whose cumulative probability exceeds the top_p threshold. A value of 1.0 considers all possible tokens. Lower values reduce the range of possible tokens, making the output less diverse.
frequency_penalty = 0.0  # Penalizes new tokens based on their existing frequency in the text so far. A positive value makes the model less likely to repeat the same lines.
presence_penalty = 0.0  # Penalizes new tokens based on whether they appear in the text so far. A positive value increases the model's likelihood to talk about new topics.

# use these parameters when making calls to the OpenAI API, or change it when needed
# for example, in client.chat.completions.create:




In [ ]:
def generate_response_with_openai(query, context="", llm_model=llm_model, your_role="" ,
      temperature=temperature,
      max_tokens=max_tokens,
      top_p=top_p,
      frequency_penalty=frequency_penalty,
      presence_penalty=presence_penalty,):
  """
  Uses OpenAI's GPT model to generate a response based on a query and optional context.

  Args:
    query (str): The main question or prompt for the model.
    context (str, optional): Additional information to help the model answer the query. Defaults to "".
    llm_model (str, optional): The name of the OpenAI language model to use (e.g., "gpt-3.5-turbo"). Defaults to the global llm_model variable.
    your_role (str, optional): The system role for the model (e.g., "You are a helpful assistant"). Defaults to "lame".
    temperature (float, optional): Controls the randomness of the output. Higher values (closer to 1) are more random. Defaults to the global temperature variable.
    max_tokens (int, optional): The maximum number of tokens the model can generate in the response. Defaults to the global max_tokens variable.
    top_p (float, optional): Controls the diversity of the output. Lower values reduce the range of possible tokens. Defaults to the global top_p variable.
    frequency_penalty (float, optional): Penalizes new tokens based on their existing frequency. Positive values make repetition less likely. Defaults to the global frequency_penalty variable.
    presence_penalty (float, optional): Penalizes new tokens based on whether they appear in the text so far. Positive values encourage new topics. Defaults to the global presence_penalty variable.

  Returns:
    str: The generated response from the OpenAI model.
  """
  prompt = f"""
  Answer the following question based on the provided context.
  Context: {context}
  Question: {query}
  """
  response = client.chat.completions.create(
      model=llm_model,
      messages=[
          {"role": "system", "content": your_role},
          {"role": "user", "content": prompt}
      ],
      temperature=temperature,
      max_tokens=max_tokens,
      top_p=top_p,
      frequency_penalty=frequency_penalty,
      presence_penalty=presence_penalty,
  )
  return response.choices[0].message.content

In [ ]:
# Test this part in line:
generate_response_with_openai ("what is RAG")

## upload the data

In [ ]:

# Upload PDF files to Googl5e Colab
uploaded = files.upload()

# Get the file paths of the uploaded files
uploaded_file_paths = list(uploaded.keys())

# Print the 4list of uploaded file paths
print("Uploaded file paths:", uploaded_file_paths)

In [ ]:
def load_pdf_documents(file_paths):
  """
  Loads PDF documents from a list of file paths.

  Args:
    file_paths (list): A list of paths to the uploaded files.

  Returns:
    list: A list of loaded documents (can be LangChain Document objects or other formats depending on the loader).
  """
  loaded_documents = []
  for file_path in file_paths:
    # Check if the file is a PDF by checking the file extension
    if file_path.lower().endswith(".pdf"):
      try:
        # Use PyPDFLoader to load the PDF
        loader = PyPDFLoader(file_path)
        documents = loader.load()
        loaded_documents.extend(documents)
        print(f"Successfully loaded PDF: {file_path}")
      except Exception as e:
        print(f"Error loading PDF {file_path}: {e}")
    else:
      print(f"Skipping non-PDF file: {file_path}")
  return loaded_documents


In [ ]:
loaded_pdfs = load_pdf_documents(uploaded_file_paths)

# You can now work with the loaded_pdfs list. For example, print the content of the first document:
if loaded_pdfs:
  print("\nContent of the first loaded document:")
  print(loaded_pdfs[0].page_content[:500]) # Print the first 500 characters
else:
  print("\nNo PDF documents were loaded.")

## Set evaluation set

In [ ]:
import json

data = [
  {
    "iPhone User Guide (iOS 3.1)": {
      "Setup and Activation": [
        {
          "question": "What operating systems are required on a Mac or PC to use the iPhone, and what are the specific version requirements for Windows and Mac OS X?",
          "ground_truth_answer": "To use the iPhone, you need a Mac or a PC with a USB 2.0 port[cite: 2345, 2346]. Required operating systems include Mac OS X version 10.4.11 or later; version 10.5.7 or later is needed for syncing Notes and for using iPhone as a modem[cite: 2346, 2347]. For Windows, Windows XP Home or Professional with Service Pack 3 or later, or Windows Vista Home Premium, Business, Enterprise, or Ultimate edition are required[cite: 2348, 2349]. Additionally, the computer must have a display resolution of 1024x768 or higher [cite: 2350], iTunes 8.2 or later (iTunes 9 or later required for certain features) [cite: 2351], and QuickTime 7.6.2 or later (for playing videos recorded by iPhone 3GS)[cite: 2352].",
          "relevant_context": "To use iPhone, you need: A Mac or a PC with a USB 2.0 port and one of the following operating systems: • Mac OS X version 10.4.11 or later; version 10.5.7 or later is required for syncing Notes and for using iPhone as a modem [cite: 2345, 2346, 2347] Windows XP Home or Professional with Service Pack 3 or later [cite: 2348] Windows Vista Home Premium, Business, Enterprise, or Ultimate edition [cite: 2349] Display resolution on your computer set to 1024x768 or higher [cite: 2350] iTunes 8.2 or later, available at www.itunes.com/download (iTunes 9 or later required for certain features) [cite: 2351] QuickTime 7.6.2 or later (for playing videos recorded by iPhone 3GS on your computer)[cite: 2352]."
        },
        {
          "question": "What is the purpose of registering the iPhone with iTunes?",
          "ground_truth_answer": "Registering the iPhone with iTunes enables iTunes to identify the iPhone when it's connected to the computer[cite: 2374]. It also helps manage the iPhone's contents, enables syncing of information from the computer and media from the iTunes library, and allows for creating backups of the iPhone’s contents and settings [cite: 2375]. iTunes also records the iPhone's serial number in the event you need it for service or in case of loss[cite: 2377].",
          "relevant_context": "Registering iPhone with iTunes enables iTunes to identify your iPhone when it's connected to your computer and help you manage its contents. [cite: 2374] You can then sync information with your computer and media from iTunes, and create backups of iPhone's contents and settings. [cite: 2375] iTunes also records iPhone's serial number in the event you need it for service or in case of loss. [cite: 2377]"
        },
        {
          "question": "How do you install the SIM card if it was not preinstalled?",
          "ground_truth_answer": "If the SIM card was not preinstalled, you must install it before you can use the iPhone[cite: 2362]. The steps are as follows: 1) Insert the end of the SIM eject tool into the hole on the SIM tray[cite: 2367]. Press firmly and push it straight in until the tray pops out[cite: 2368]. 2) Pull out the SIM tray and place the SIM card in the tray[cite: 2370]. The angled corner of the SIM ensures that the card fits only the correct way in the tray[cite: 2371]. 3) With the tray aligned as shown, carefully replace the SIM tray containing the SIM card in iPhone[cite: 2372].",
          "relevant_context": "If your SIM card was not preinstalled, you must install the SIM card before you can use iPhone. [cite: 2362] Install the SIM card: 1 Insert the end of the SIM eject tool into the hole on the SIM tray. [cite: 2367] Press firmly and push it straight in until the tray pops out. [cite: 2368] 2 Pull out the SIM tray and place the SIM card in the tray. [cite: 2370] The angled corner of the SIM ensures that the card fits only the correct way in the tray. [cite: 2371] 3 With the tray aligned as shown, carefully replace the SIM tray containing the SIM card in iPhone. [cite: 2372]"
        },
        {
          "question": "How do you turn the iPhone completely off and then back on?",
          "ground_truth_answer": "To turn the iPhone completely off, press and hold the Sleep/Wake button for a few seconds until the red slider appears, then drag the slider[cite: 2690]. When the iPhone is off, incoming calls go straight to voicemail[cite: 2690]. To turn the iPhone back on, press and hold the Sleep/Wake button until the Apple logo appears[cite: 2690].",
          "relevant_context": "Turn iPhone completely off: Press and hold the Sleep/Wake button for a few seconds until the red slider appears, then drag the slider. [cite: 2690] When iPhone is off, incoming calls go straight to voicemail. [cite: 2690] Turn iPhone on: Press and hold the Sleep/Wake button until the Apple logo appears. [cite: 2690]"
        }
      ],
      "Features and Functionality": [
        {
          "question": "What applications are included with the iPhone?",
          "ground_truth_answer": "The iPhone includes the following applications: Phone, Mail, Safari, iPod, Messages, Calendar, Photos, Camera, YouTube, Stocks, Maps, Weather, Voice Memos, Notes, Clock, Calculator, iTunes Store, App Store, Compass, and Contacts[cite: 2653, 2657, 2659].",
          "relevant_context": "The following applications are included with iPhone: Phone, Mail, Safari, iPod, Messages, Calendar, Photos, Camera, YouTube, Stocks, Maps, Weather, Voice Memos, Notes, Clock, Calculator, iTunes Store, App Store, Compass, and Contacts. [cite: 2653, 2657, 2659]"
        },
        {
          "question": "How can you browse album covers using Cover Flow on the iPhone?",
          "ground_truth_answer": "When you are Browse music, you can rotate the iPhone sideways to see your iTunes content in Cover Flow and browse music by album artwork[cite: 3811]. You can browse album covers by dragging or flicking left or right[cite: 3826].",
          "relevant_context": "When you're Browse music, you can rotate iPhone sideways to see your iTunes content in Cover Flow and browse your music by album artwork. [cite: 3811] Browse album covers: Drag or flick left or right. [cite: 3826]"
        },
        {
          "question": "How can you copy and paste text on the iPhone using the onscreen keyboard?",
          "ground_truth_answer": "To copy text, first select the desired text, then tap 'Copy'[cite: 2857]. To paste text, tap the insertion point and tap 'Paste'. The last text that was cut or copied is inserted[cite: 2863]. Alternatively, you can select text and tap 'Paste' to replace the selected text[cite: 2864].",
          "relevant_context": "Cut or copy text: Select text, then tap Cut or Copy. [cite: 2857] Paste text: Tap the insertion point and tap Paste. [cite: 2863] Or select text and tap Paste to replace the text. [cite: 2864]"
        },
        {
          "question": "What is the purpose of the Ring/Silent Switch, and does it silence all sounds, including Clock alarms?",
          "ground_truth_answer": "The Ring/Silent switch is used to change between ring and silent modes[cite: 2706]. When set to silent mode, the iPhone doesn't ring or play alerts and other sound effects[cite: 2709]. However, Clock alarms still sound even if you set the Ring/Silent switch to silent[cite: 2710, 3302].",
          "relevant_context": "Flip the Ring/Silent switch to change between ring and silent modes. [cite: 2706] When set to silent mode, iPhone doesn't ring or play alerts and other sound effects. [cite: 2709] Important: Clock alarms, audio-dependent applications such as iPod, and many games still play sounds through the built-in speaker when iPhone is set to silent mode. [cite: 2710] Important: Clock alarms still sound even if you set the Ring/Silent switch to silent. [cite: 3302]"
        },
        {
          "question": "How do you configure the iPhone to prevent automatic syncing when connected to a computer?",
          "ground_truth_answer": "To prevent automatic syncing, connect the iPhone to your computer. In iTunes, select iPhone in the sidebar, then click the 'Summary' tab. Deselect 'Automatically sync when this iPhone is connected'[cite: 2458, 2517, 2518]. You can also prevent automatic syncing for all iPhones and iPods by going to iTunes preferences (iTunes > Preferences on a Mac or Edit > Preferences on a PC), clicking 'Devices', and selecting 'Disable automatic syncing for all iPhones and iPods'[cite: 2522].",
          "relevant_context": "Prevent automatic syncing for iPhone: Connect iPhone to your computer. [cite: 2517] In iTunes, select iPhone in the sidebar, then click the Summary tab. [cite: 2518] Prevent automatic syncing for all iPhones and iPods: In iTunes choose iTunes > Preferences (on a Mac) or Edit > Preferences (on a PC), click Devices, and select 'Disable automatic syncing for all iPhones and iPods.' [cite: 2522] By default, 'Automatically sync when this iPhone is connected' is selected. [cite: 2440]"
        },
        {
          "question": "What is the process for setting up a MobileMe or Microsoft Exchange account directly on the iPhone?",
          "ground_truth_answer": "To set up a MobileMe or Microsoft Exchange account directly on the iPhone, from the Home screen, tap 'Settings'[cite: 2540]. Then, tap 'Mail, Contacts, Calendars,' and tap 'Add Account'[cite: 2541]. Tap 'Microsoft Exchange' or 'MobileMe' (or other options such as Google email, Yahoo! Mail, or AOL)[cite: 2543, 2544, 2545, 2546, 2547, 2550]. Enter your account information and tap 'Save'[cite: 2551]. If setting up a MobileMe or Microsoft Exchange account, tap the items you want to use on iPhone—mail, contacts, calendars, and bookmarks (MobileMe only)[cite: 2558].",
          "relevant_context": "Set up an account on iPhone: 1 From the iPhone Home screen, tap Settings. [cite: 2540] 2 Tap 'Mail, Contacts, Calendars,' then tap Add Account. [cite: 2541] 3 Tap an account type: Microsoft Exchange [cite: 2543] MobileMe [cite: 2544] Google email [cite: 2545] Yahoo! Mail [cite: 2548] AOL [cite: 2549] Other [cite: 2550] 4 Enter your account information and tap Save. [cite: 2551] 5 If you're setting up a MobileMe or Microsoft Exchange account, tap the items you want to use on iPhone-mail, contacts, calendars, and bookmarks (MobileMe only). [cite: 2558]"
        },
        {
          "question": "How does the iPhone handle incoming calls if a user is actively transferring data over a cellular network without a 3G connection?",
          "ground_truth_answer": "If a user is actively transferring data over a cellular network (such as downloading a webpage) and does not have a 3G connection, they may not be able to receive calls[cite: 2985]. Incoming calls then go directly to voicemail[cite: 2986]. You must have a Wi-Fi connection to use Internet applications while also talking on the phone[cite: 2967].",
          "relevant_context": "If you don't have a 3G connection, you may not be able to receive calls while iPhone is actively transferring data over a cellular network-downloading a webpage, for example. [cite: 2985] Incoming calls then go directly to voicemail. [cite: 2986] You must have a Wi-Fi connection to use Internet applications while also talking on the phone. [cite: 2967]"
        }
      ],
      "Accessibility and Troubleshooting": [
        {
          "question": "Describe the function of Voice Control on the iPhone 3GS and how it is activated.",
          "ground_truth_answer": "Voice Control (available only on iPhone 3GS) allows users to make phone calls and control iPod music playback using voice commands[cite: 2915]. It is activated by pressing and holding the Home button until the Voice Control screen appears and a beep is heard[cite: 2922]. Alternatively, you can press and hold the center button on the iPhone headset (or an equivalent button on a Bluetooth headset)[cite: 2923].",
          "relevant_context": "Voice Control (available only on iPhone 3GS) lets you make phone calls and control iPod music playback using voice commands. [cite: 2915] Use Voice Control: Press and hold the Home button until the Voice Control screen appears and you hear a beep. [cite: 2922] You can also press and hold the center button on the iPhone headset (or the equivalent button on your Bluetooth headset). [cite: 2923]"
        },
        {
          "question": "How do you mute or unmute VoiceOver using a gesture?",
          "ground_truth_answer": "To mute VoiceOver, use a three-finger double tap[cite: 5499]. To unmute VoiceOver, use a three-finger double tap again to turn speaking back on[cite: 5499].",
          "relevant_context": "Three-finger double tap: Mute or unmute VoiceOver. [cite: 5499]"
        },
        {
          "question": "What are the steps to force quit an application that is not responding?",
          "ground_truth_answer": "To force quit an application, press and hold the Sleep/Wake button on top of iPhone for a few seconds until a red slider appears, and then press and hold the Home button until the application quits[cite: 3101].",
          "relevant_context": "Force quit an application: Press and hold the Sleep/Wake button on top of iPhone for a few seconds until a red slider appears, and then press and hold the Home button until the application quits. [cite: 3101]"
        },
        {
          "question": "How do you force restart the iPhone if it is frozen and unresponsive?",
          "ground_truth_answer": "To force restart the iPhone if it is frozen and unresponsive, press and hold the Sleep/Wake button and the Home button at the same time for at least ten seconds, until the Apple logo appears[cite: 3102].",
          "relevant_context": "Reset iPhone: Press and hold the Sleep/Wake button and the Home button at the same time for at least ten seconds, until the Apple logo appears. [cite: 3102]"
        }
      ]
    }
  },
  {
    "Android 3.0 User's Guide": {
      "Setup and Configuration": [
        {
          "question": "How do you back up and restore data using a Google Account during the initial setup?",
          "ground_truth_answer": "You can use your Google Account to back up your apps, settings (such as bookmarks and Wi-Fi passwords), and other data[cite: 252]. If you previously backed up data with that account, you can choose to restore it to the tablet now[cite: 253]. You must restore data during the initial setup, as data cannot be restored after setup is complete[cite: 247, 248].",
          "relevant_context": "You can use your Google Account to back up your apps, settings (such as bookmarks and Wi-Fi passwords), and other data. [cite: 252] If you previously backed up your data with this account, you can restore it to this device now. [cite: 253] If you want to restore your settings, you must do that now during setup. [cite: 247] You can't restore data after setup is complete. [cite: 248]"
        },
        {
          "question": "How do you configure the tablet to prevent data transmission over other carriers' mobile networks when roaming?",
          "ground_truth_answer": "To configure the tablet to prevent data transmission over other carriers' mobile networks when roaming, open the Settings application[cite: 1205]. Touch 'Wireless & networks' > 'Mobile networks' and uncheck 'Data roaming'[cite: 1206]. When 'Data roaming' is unchecked, you can still transmit data via a Wi-Fi connection[cite: 1207].",
          "relevant_context": "Open the Settings application. [cite: 1205] Touch Wireless & networks > Mobile networks and uncheck Data roaming. [cite: 1206] With Data roaming unchecked, you can still transmit data with a Wi-Fi connection. [cite: 1207]"
        },
        {
          "question": "What types of data are backed up automatically when you use the Google Account backup service?",
          "ground_truth_answer": "When using the Google Account backup service, a wide variety of personal data is backed up, including: your Android settings, such as Wi-Fi passwords and user dictionary [cite: 258, 259]; many Google application settings, such as Browser bookmarks [cite: 259]; and apps downloaded from Android Market[cite: 260]. Some third-party apps may also take advantage of this service[cite: 264].",
          "relevant_context": "Many kinds of data are backed up, including: Your Android settings, such as your Wi-Fi networks and passwords, user diction-ary, and so on[cite: 257, 258]; Many Google application settings, such as your Browser bookmarks[cite: 259]; The apps you download from Android Market[cite: 260]. Some third-party apps also make use of the backup service, so if you reinstall one, its settings and data are restored. [cite: 264]"
        },
        {
          "question": "How do you secure the Android tablet's lock screen?",
          "ground_truth_answer": "To secure the tablet's lock screen, open Settings and touch 'Location & security'[cite: 522]. Then, touch 'Configure lock screen' [cite: 523], and select 'Pattern', 'PIN', or 'Password'[cite: 524].",
          "relevant_context": "1 Touch the time in the System bar, touch the time in the Status Details panel, and touch Settings at the bottom of the Quick Settings. [cite: 520] 2 Touch Location & security. [cite: 522] 3 Touch Configure lock screen. [cite: 523] 4 Touch Pattern, PIN, or Password. [cite: 524]"
        }
      ],
      "Features and Usage": [
        {
          "question": "What is the difference between swiping and dragging on the Android touchscreen?",
          "ground_truth_answer": "Dragging involves touching and holding an item for a moment and then, without lifting your finger, moving it on the screen to the target position[cite: 335]. Swiping or sliding involves quickly moving your finger across the surface of the screen, without pausing when you first touch it[cite: 337].",
          "relevant_context": "Drag: Touch & hold an item for a moment and then, without lifting your finger, move your finger on the screen until you reach the target position. [cite: 335] Swipe or slide: To swipe or slide, you quickly move your finger across the surface of the screen, without pausing when you first touch it (so you don't drag an item instead). [cite: 337]"
        },
        {
          "question": "How can you optimize battery life by turning off specific features?",
          "ground_truth_answer": "You can extend battery life by turning off radios that you are not using, such as Wi-Fi, Bluetooth, or GPS[cite: 643, 644]. You should also turn down screen brightness and set a shorter screen timeout[cite: 646, 647]. Additionally, turning off automatic syncing for apps like Gmail, Calendar, and Contacts can help optimize battery life[cite: 648, 649].",
          "relevant_context": "You can extend your battery's life between charges by turning off features that you don't need. [cite: 641] Turn off radios that you aren't using. [cite: 643] If you aren't using Wi-Fi, Bluetooth, or GPS, use the Settings app to turn them off. [cite: 644] Turn down screen brightness and set a shorter screen timeout. [cite: 646, 647] If you don't need them, turn off automatic syncing for Gmail, Calendar, Contacts, and other apps. [cite: 648, 649]"
        },
        {
          "question": "How do you open the Apps screen and add a shortcut for an app to a Home screen?",
          "ground_truth_answer": "To open the Apps screen, touch the Apps button (at the top right) of any Home screen[cite: 694]. To add a shortcut for an app to a Home screen, touch and hold the app on the Apps screen until outlines of the Home screens appear, then drag the app icon to the Home screen[cite: 696, 697]. If you pause before releasing your finger on a Home screen, that Home screen enlarges, allowing you to place the shortcut where you want it[cite: 698].",
          "relevant_context": "In Home, touch the Apps button (at the top right) of any Home screen to open the Apps screen. [cite: 694] Drag an app down so you can add a shortcut for it to a Home screen. [cite: 385] You can add shortcut to an app to a Home screen by touching & holding the app on the Apps screen until outlines of the Home screens appear below the app icons. [cite: 696] Then drag the app icon to a the Home screen. [cite: 697] If you pause before releasing your finger on a Home screen, that Home screen enlarges, so you can place the shortcut where you want it. [cite: 698]"
        },
        {
          "question": "Describe how to connect a Macintosh computer to the Android tablet for file transfer, including any necessary software.",
          "ground_truth_answer": "To connect an Android tablet to a Macintosh computer for file transfer, you must first install the free Android File Transfer application on the Macintosh, as Mac OS X does not natively support the MTP protocol[cite: 1406]. The application is available from http://www.android.com/filetransfer[cite: 1407]. Once installed, connect the tablet to the computer using the USB cable that came with the tablet[cite: 1413]. Android File Transfer will start and open a window displaying the contents of your tablet, allowing you to view and copy files[cite: 1414].",
          "relevant_context": "You can connect your tablet to a computer running Mac OS X with a USB cable, to transfer music, pictures, and other files between your tablet and the computer. [cite: 1405] This connection uses the MTP protocol, which is not supported by Mac OS X natively, so you first need to install the free Android File Transfer application on your Macintosh. [cite: 1406] It's available from http://www.android.com/filetransfer, along with instructions on how to install it. [cite: 1407] Once you have installed Android File Transfer on your Macintosh, you can connect your tablet to your computer using the USB cable that came with your tablet, so you can view and copy files. [cite: 1410] Android File Transfer starts, if it's not already running, and opens a window that displays the contents of your tablet. [cite: 1414]"
        },
        {
          "question": "How do you manage notifications on the tablet, and what steps are required to open the Notifications list?",
          "ground_truth_answer": "Notifications report the arrival of new messages, calendar events, alarms, and ongoing events, such as when you have configured your tablet as a Wi-Fi hotspot[cite: 417]. To open the Notifications list, you must touch the time in the System bar[cite: 425].",
          "relevant_context": "Notifications report the arrival of new messages, calendar events, and alarms, as well as ongoing events, such as when you have configured your tablet as a Wi-Fi hotspot. [cite: 417] 1 Touch the time in the System bar. [cite: 425]"
        },
        {
          "question": "How do you enter text by speaking, and what is required to use this feature?",
          "ground_truth_answer": "To enter text by speaking, touch a text field or a location in text you've already entered in a text field, then touch the Microphone key on the onscreen keyboard[cite: 1002, 1003]. When prompted to 'Speak now', speak what you want to enter[cite: 1004]. Text that is entered by speaking is underlined[cite: 1007]. This feature is experimental and uses Google's speech-recognition service, so you must have a data connection on a mobile or Wi-Fi network to use it[cite: 1000].",
          "relevant_context": "You can enter text by speaking, in most places that you can enter text with the onscreen keyboard. [cite: 1001] 1 Touch a text field, or a location in text you've already entered in a text field. [cite: 1002] 2 Touch the Microphone key on the onscreen keyboard. [cite: 1003] 3 When prompted to 'Speak now,' speak what you want to enter. [cite: 1004] Text that you enter by speaking is underlined. [cite: 1007] Voice input is an experimental feature that uses Google's speech-recognition service, so you must have a data connection on a mobile or Wi-Fi network to use it. [cite: 1000]"
        },
        {
          "question": "What are the steps to set up the Android tablet as a portable Wi-Fi hotspot?",
          "ground_truth_answer": "To set up the Android tablet as a portable Wi-Fi hotspot, open the Settings application[cite: 1499]. Touch 'Wireless & networks' > 'Tethering & portable hotspot'[cite: 1501]. Check 'Portable Wi-Fi hotspot'[cite: 1502]. The tablet will start broadcasting its Wi-Fi network name (SSID), so you can connect to it with up to 8 computers or other devices[cite: 1504]. An ongoing notification is added to the System bar, to the left of the time[cite: 1505].",
          "relevant_context": "1 Open the Settings application. [cite: 1499] 2 Touch Wireless & networks > Tethering & portable hotspot. [cite: 1501] 3 Check Portable Wi-Fi hotspot. [cite: 1502] After a moment, the tablet starts broadcasting its Wi-Fi network name (SSID), so you can connect to it with up to 8 computers or other devices. [cite: 1504] An ongoing notification is added to the System bar, to the left of the time. [cite: 1505]"
        }
      ],
      "Google Search and Voice Actions": [
        {
          "question": "How do you use Voice Actions to set an alarm or send an email?",
          "ground_truth_answer": "To set an alarm, you can say 'Set alarm' followed by a time, such as '10:45am', and an optional label. Clock opens, prompting you to confirm the details before setting the alarm[cite: 1811]. To send an email, say 'Send email' followed by 'To' and the contact name, 'Subject' and the subject, and 'Message' and the message you want to send. Gmail opens where you can complete the message before sending it[cite: 1807].",
          "relevant_context": "Set alarm: Say 'Set alarm' followed by a time, such as '10:45am' or '20 minutes from now', and an optional label. [cite: 1811] Clock opens, prompting you to confirm the details before setting the alarm. [cite: 1811] Send email: Say 'Send email' followed by 'To' one or more contacts, 'Cc' one or more contacts, 'Bcc' one or more contacts, 'Subject' followed by a subject, and 'Message' followed by the message you want to send. [cite: 1807] Gmail opens where you can complete the message before sending it. [cite: 1807]"
        },
        {
          "question": "What happens when network status icons turn blue?",
          "ground_truth_answer": "Network status icons turn blue if you have a Google Account added to your tablet and the tablet is connected to Google services[cite: 407, 1187]. This allows for syncing Gmail, Calendar events, contacts, and backing up settings[cite: 407, 1187].",
          "relevant_context": "Network status icons turn blue if you have a Google Account added to your tablet and the tablet is connected to Google services, for syncing your Gmail, Calendar events, contacts, for backing up your settings, and so on. [cite: 407, 1187]"
        },
        {
          "question": "How do you search the web using Voice Search?",
          "ground_truth_answer": "To search the web using Voice Search, return to the Home screen [cite: 1769], and touch the Microphone button at the top-left of the screen[cite: 1770]. When prompted to 'Speak now', speak the words you want to search for on the web[cite: 1779, 1783]. When you're finished speaking, Voice Search analyzes your speech and Browser opens, displaying results of a Google Search on the web[cite: 1786].",
          "relevant_context": "1 Return Home. [cite: 1769] 2 Touch the Microphone button at the top-left of the screen. [cite: 1770] Speak now. [cite: 1779] 3 Speak the words you want to search for on the web. [cite: 1783] When you're finished speaking, Voice Search analyzes your speech and Browser opens, displaying results of a Google Search on the web. [cite: 1786]"
        }
      ]
    }
  },
  {
    "Samsung Galaxy S23 User Manual": {
      "Setup and Charging": [
        {
          "question": "How do you turn the Samsung Galaxy S23 device on and off?",
          "ground_truth_answer": "To turn the device on, press and hold the Side key for a few seconds[cite: 6272]. To turn the device off, simultaneously press and hold the Side key and the Volume Down key[cite: 6276]. Alternatively, open the notification panel, swipe downwards, and tap the power icon [cite: 6277], then tap 'Power off'[cite: 6278].",
          "relevant_context": "Press and hold the Side key for a few seconds to turn on the device. [cite: 6272] To turn off the device, press and hold the Side key and the Volume Down key simultaneously. [cite: 6276] Alternatively, open the notification panel, swipe downwards, and then tap[cite: 6277]. Tap Power off. [cite: 6278]"
        },
        {
          "question": "How can you charge the Samsung Galaxy S23 battery using wireless power sharing with another device?",
          "ground_truth_answer": "To charge another device using Wireless power sharing, open the notification panel, swipe downwards, and then tap the Wireless power sharing icon to turn it on[cite: 6149]. Place the other device on the center of your phone, with their backs facing each other[cite: 6151]. When finished charging, disconnect the other device from your phone[cite: 6156].",
          "relevant_context": "Open the notification panel, swipe downwards, and then tap (Wireless power sharing) to turn it on. [cite: 6149] Place the other device on the centre of your phone, with their backs facing. [cite: 6151] When you are finished charging, disconnect the other device from your phone. [cite: 6156]"
        },
        {
          "question": "What precautions should be taken when using wireless charging to avoid damaging the device or cards?",
          "ground_truth_answer": "When using wireless charging, do not place the device on the wireless charger with a credit card or radio-frequency identification (RFID) card between the back of the device and the device cover[cite: 6128, 6129]. Avoid placing conductive materials, such as metal objects and magnets, between the device and the wireless charger[cite: 6130]. Ensure you use only Samsung-approved wireless chargers[cite: 6132].",
          "relevant_context": "Do not place the device on the wireless charger with a credit card or radio-frequency identification (RFID) card (such as a transportation card or a key card) placed between the back of the device and the device cover. [cite: 6128, 6129] Do not place the device on the wireless charger when conductive materials, such as metal objects and magnets, are placed between the device and the wireless charger. [cite: 6130] Use Samsung-approved wireless chargers. [cite: 6132]"
        },
        {
          "question": "How do you force restart the device if it is frozen and unresponsive?",
          "ground_truth_answer": "If the device is frozen and unresponsive, press and hold the Side key and the Volume Down key simultaneously for more than 7 seconds to restart it[cite: 6284].",
          "relevant_context": "If your device is frozen and unresponsive, press and hold the Side key and the Volume Down key simultaneously for more than 7 seconds to restart it. [cite: 6284]"
        },
        {
          "question": "How can you insert a nano-SIM card into the Samsung Galaxy S23 Ultra?",
          "ground_truth_answer": "To insert a nano-SIM card, insert the ejection pin into the hole on the tray to loosen the tray[cite: 6239]. Gently pull out the tray from the tray slot[cite: 6240]. Place the SIM or USIM card on the tray with the gold-colored contacts facing upwards and gently press the SIM or USIM card into the tray to secure it[cite: 6241]. Finally, insert the tray back into the tray slot[cite: 6242].",
          "relevant_context": "1 Insert the ejection pin into the hole on the tray to loosen the tray. [cite: 6239] 2 Pull out the tray gently from the tray slot. [cite: 6240] 3 Place the SIM or USIM card on the tray with the gold-coloured contacts facing upwards and gently press the SIM or USIM card into the tray to secure it. [cite: 6241] 4 Insert the tray back into the tray slot. [cite: 6242]"
        }
      ],
      "Features and Usage": [
        {
          "question": "How do you hide the navigation bar on the Samsung Galaxy S23?",
          "ground_truth_answer": "To hide the navigation bar, open 'Settings', tap 'Display' → 'Navigation bar', and then tap 'Swipe gestures' under 'Navigation type'[cite: 6394].",
          "relevant_context": "Open Settings, tap Display → Navigation bar, and then tap Swipe gestures under Navigation type. [cite: 6394]"
        },
        {
          "question": "Describe the steps to capture a screenshot on the Samsung Galaxy S23 using key capture and swipe capture methods.",
          "ground_truth_answer": "To capture a screenshot using the key capture method, simultaneously press the Side key and the Volume Down key[cite: 6546]. To capture a screenshot using the swipe capture method, swipe your hand to the left or right across the screen with the edge of your hand[cite: 6547]. You can check the captured screenshots in Gallery[cite: 6545].",
          "relevant_context": "Method 1) Key capture: Press the Side key and the Volume Down key simultaneously. [cite: 6546] Method 2) Swipe capture: With the edge of your hand, swipe your hand to the left or right across the screen. [cite: 6547] You can check the captured screenshots in Gallery. [cite: 6545]"
        },
        {
          "question": "How do you turn on the \"Show Apps screen button on Home screen\" feature?",
          "ground_truth_answer": "To turn on the 'Show Apps screen button on Home screen' feature, touch and hold an empty area on the Home screen, tap 'Settings', and then tap the 'Show Apps screen button on Home screen' switch to turn it on[cite: 6416].",
          "relevant_context": "On the Home screen, touch and hold an empty area, tap Settings, and then tap the Show Apps screen button on Home screen switch to turn it on. [cite: 6416]"
        },
        {
          "question": "What is the process for creating a new contact on the Samsung Galaxy S23?",
          "ground_truth_answer": "To create a new contact, open the Contacts app and tap '+'[cite: 7093]. Select a storage location [cite: 7094], enter the contact information, and then tap 'Save'[cite: 7095].",
          "relevant_context": "1 Open the Contacts app and tap +. [cite: 7093] 2 Select a storage location. [cite: 7094] 3 Enter contact information and tap Save. [cite: 7095]"
        },
        {
          "question": "What is the purpose of the Secure Folder feature, and how do you set it up?",
          "ground_truth_answer": "Secure Folder protects your private content and apps, such as pictures and contacts, from being accessed by others[cite: 8734]. It is a separate, secured storage area where data cannot be transferred to other devices through unapproved sharing methods, such as USB or Wi-Fi Direct[cite: 8736]. To set it up, open Settings and tap 'Security and privacy' -> 'Secure Folder' [cite: 8739], and follow the on-screen instructions to complete the setup[cite: 8740].",
          "relevant_context": "Secure Folder protects your private content and apps, such as pictures and contacts, from being accessed by others. [cite: 8734] Secure Folder is a separate, secured storage area. [cite: 8736] 1 Open Settings and tap Security and privacy → Secure Folder. [cite: 8739] 2 Follow the on-screen instructions to complete the setup. [cite: 8740]"
        },
        {
          "question": "How do you change the screen mode and adjust the display color on the device?",
          "ground_truth_answer": "To change the screen mode, open Settings, tap 'Display' → 'Screen mode' and select a mode (Vivid or Natural)[cite: 8584, 8585, 8587]. To adjust the display color (only available in Vivid mode)[cite: 8588], tap 'Advanced settings'[cite: 8596], and adjust the R (Red), G (Green), or B (Blue) color bars to your preference[cite: 8597].",
          "relevant_context": "On the Settings screen, tap Display → Screen mode and select a mode you want. [cite: 8584] Vivid: This optimises the colour range, saturation, and sharpness of your display. [cite: 8585] Natural: This adjusts the screen to a natural tone. [cite: 8587] You can adjust the display colour only in Vivid mode. [cite: 8588] 1 On the Settings screen, tap Display → Screen mode → Vivid. [cite: 8595] 2 Tap Advanced settings. [cite: 8596] 3 Adjust the R (Red), G (Green), or B (Blue) colour bar to your preference. [cite: 8597]"
        },
        {
          "question": "How do you manage apps using the Dual Messenger feature?",
          "ground_truth_answer": "The Dual Messenger feature allows you to install a second app and use two separate accounts for the same messenger app[cite: 8874]. To use it, open Settings, tap 'Advanced features' → 'Dual Messenger'[cite: 8875], and tap the switch of an app to install the second app[cite: 8876]. The second app's icon will be displayed with a specific badge[cite: 8877]. To uninstall the second app, tap the switch of the app you want to uninstall and tap 'Uninstall'[cite: 8880].",
          "relevant_context": "Install the second app and use two separate accounts for the same messenger app. [cite: 8874] 1 On the Settings screen, tap Advanced features → Dual Messenger. [cite: 8875] 2 Tap the switch of an app to install the second app. [cite: 8876] The second app’s icon will be displayed with. [cite: 8877] 1 On the Settings screen, tap Advanced features → Dual Messenger. [cite: 8879] 2 Tap the switch of the app you want to uninstall and tap Uninstall. [cite: 8880]"
        }
      ],
      "Camera and S Pen (Galaxy S23 Ultra)": [
        {
          "question": "How do you use the S Pen to control apps remotely via Air actions?",
          "ground_truth_answer": "Air actions allow you to control apps remotely by using the S Pen connected through Bluetooth Low Energy (BLE)[cite: 6689]. The S Pen must be charged before you can use the Air actions feature[cite: 6693]. To enable this feature, open Settings, tap 'Advanced features' → 'S Pen' → 'Air actions', and then tap the switch to turn it on[cite: 6700]. You can control apps remotely by pressing and holding the S Pen button[cite: 6690].",
          "relevant_context": "Control apps remotely by using the S Pen connected to your device through Bluetooth Low Energy (BLE). [cite: 6689] Use your S Pen as a remote control to open apps, take pictures, play movies, and more without touching the screen. [cite: 6715] For example, you can open apps, such as the camera app, by pressing and holding the S Pen button. [cite: 6690] The S Pen must be charged before you can use the Air actions feature. [cite: 6693] 1 Open Settings, tap Advanced features → S Pen → Air actions, and then tap the switch to turn on the feature. [cite: 6700]"
        },
        {
          "question": "How do you use the Camera app to take pictures?",
          "ground_truth_answer": "To take pictures, open the Camera app[cite: 7176]. Tap the image on the preview screen where the camera should focus[cite: 7181]. To take a picture, tap the camera button[cite: 7183].",
          "relevant_context": "1 Open the Camera app. [cite: 7176] 2 Tap the image on the preview screen where the camera should focus. [cite: 7181] 3 Tap to take a picture. [cite: 7183]"
        },
        {
          "question": "How can you trim a voice memo using the Voice Recorder app?",
          "ground_truth_answer": "To trim a voice memo, on the Voice Memos screen, tap the button next to the memo you want to trim[cite: 4695]. Tap 'Trim Memo'[cite: 4696]. Using the time markers as a guide, drag the edges of the audio region to adjust the beginning and end of the voice memo[cite: 4696]. To preview your edit, tap the play button[cite: 4697]. To execute the trim, select 'Trim Voice Memo' and double-tap[cite: 4698].",
          "relevant_context": "On the Voice Memos screen, tap next to the memo you want to trim. [cite: 4695] 2 Tap Trim Memo. [cite: 4696] 3 Using the time markers as a guide, drag the edges of the audio region to adjust the beginning and end of the voice memo. [cite: 4696] To preview your edit, tap[cite: 4697]. 4 Tap Trim Voice Memo. [cite: 4698]"
        },
        {
          "question": "How do you record a fun video with virtual handwriting or drawings using the AR Doodle feature?",
          "ground_truth_answer": "To record a fun video with virtual handwriting or drawings using AR Doodle, open the AR Zone app and tap 'AR Doodle'[cite: 7769]. When the camera recognizes the subject, the recognition area appears[cite: 7770]. Write or draw in the recognition area[cite: 7771]. Tap the record button to record a video [cite: 7782], and tap the stop button when finished[cite: 7784]. When the camera recognizes a face or space, the doodles will follow the face as it moves or be fixed in the same place even if the camera moves[cite: 7768].",
          "relevant_context": "Record fun videos with virtual handwriting or drawings on people or pets' (cat and dogs) faces or anywhere else. [cite: 7767] When the camera recognises a face or space, the doodles on the face will follow the face as it moves, and the doodles in the space will be fixed in the same place even if the camera moves. [cite: 7768] 1 Open the AR Zone app and tap AR Doodle. [cite: 7769] When the camera recognises the subject, the recognition area will appear on the screen. [cite: 7770] 2 Write or draw in the recognition area. [cite: 7771] 3 Tap [cite: 7782] to record a video. [cite: 7782] 4 Tap [cite: 7784] to stop recording the video."
        },
        {
          "question": "How do you activate the S Pen unlock feature?",
          "ground_truth_answer": "To activate the S Pen unlock feature, open Settings and tap 'Advanced features' → 'S Pen' → 'More S Pen settings'[cite: 7005]. Tap the 'S Pen unlock' switch to turn it on[cite: 7006]. Now, you can unlock the screen by pressing the S Pen button[cite: 7007].",
          "relevant_context": "1 Open Settings and tap Advanced features → S Pen → More S Pen settings. [cite: 7005] 2 Tap the S Pen unlock switch to turn it on. [cite: 7006] Now you can unlock the screen by pressing the S Pen button. [cite: 7007]"
        },
        {
          "question": "How do you delete registered fingerprint data on the Samsung Galaxy S23?",
          "ground_truth_answer": "To delete registered fingerprint data, open Settings, tap 'Security and privacy' → 'Biometrics' → 'Fingerprints'[cite: 8705, 8706]. Unlock the screen using the preset screen lock method[cite: 8706]. Select the fingerprint to delete and tap 'Remove'[cite: 8706].",
          "relevant_context": "1 On the Settings screen, tap Security and privacy → Biometrics → Fingerprints. [cite: 8705, 8706] 2 Unlock the screen using the preset screen lock method. [cite: 8706] 3 Select a fingerprint to delete and tap Remove. [cite: 8706]"
        }
      ]
    }
  }
]

# Lets build the RAG pipeline - Data Preparation
The following steps outline the process for building and evaluating a RAG pipeline using Python libraries like LangChain and LangGraph in a Google Colab environment.

## Chunking (Text Splitting)

Since LLMs have a limited context window and large documents are hard to search, we need to break the documents into smaller, manageable chunks. The RecursiveCharacterTextSplitter is recommended for general text use cases, as it recursively splits the text based on common separators (like new lines) to maintain semantic relationships within the chunks.



In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
def chunk_documents(loaded_pdfs, chunking_type="RecursiveCharacterTextSplitter", chunk_size=1000, chunk_overlap=200):
  """
  Chunks a list of loaded documents using a specified chunking method.

  Args:
    loaded_pdfs (list): A list of loaded documents (e.g., LangChain Document objects).
  52  chunking_type (str, optional): The type of text splitter to use.
                                     Defaults to "RecursiveCharacterTextSplitter".
                                     Currently only supports "RecursiveCharacterTextSplitter".
    chunk_size (int, optional): The maximum size of each chunk. Defaults to 1000.
    chunk_overlap (int, optional): The number of characters to overlap between chunks.
                                   Defaults to 200.

  Returns:
    list: A list of text chunks.
  """
  if chunking_type == "RecursiveCharacterTextSplitter":
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(loaded_pdfs)
  else:
    raise ValueError(f"Unsupported chunking type: {chunking_type}")

  return chunks



In [ ]:
# Example usage with the loaded_pdfs from the previous cell:
if loaded_pdfs:
  text_chunks = chunk_documents(loaded_pdfs)
  print(f"\nCreated {len(text_chunks)} chunks.")
  print("\nFirst chunk content:")
  print(text_chunks[0].page_content[:500])
else:
  print("\nNo loaded PDFs to chunk.")
12

## Embedding and Vector Storage
Next, we convert each text chunk into a numerical vector (embedding) using an embedding model. These embeddings capture the semantic meaning of the text. We then store these embeddings and their corresponding text chunks in a Vector Store (like ChromaDB) for efficient similarity search.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


In [ ]:
# Initialize the embedding model (e.g., OpenAIEmbeddings or a local model)
# Note: You will need an API key for OpenAI
embeddings = OpenAIEmbeddings(api_key=personal_api_key)

# Create a Vector Store from the chunks
vector_store = Chroma.from_documents(
    text_chunks,
    embeddings
)

# You can save the vector store to disk for later use
# vector_store.persist()

# Retrieval and Generation (RAG Chain)

 ## Setting up the RAG Chain
When a user asks a question, the RAG pipeline retrieves relevant information from the Vector Store (retrieval step) and combines it with the user's query before sending it to the LLM (augmentation and generation steps).

In [ ]:
from langchain.chains import RetrievalQA

# Define the retriever
retriever = vector_store.as_retriever()


In [ ]:
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model with your API key and desired parameters
llm = ChatOpenAI(
    model=llm_model,
    temperature=temperature,
    max_tokens=max_tokens,
    top_p=top_p,
    frequency_penalty=frequency_penalty,
    presence_penalty=presence_penalty,
    openai_api_key=personal_api_key # Pass your personal_api_key here
)

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# Now you can use the qa_chain to answer questions


In [ ]:
query = "What is iphone?"
response = qa_chain.invoke(query)
print(response)

## Testing the RAG Pipeline

In [ ]:
query = "How do you install the SIM card in the iPhone?"
response = qa_chain.invoke(query)
print(response)

# Evaluation with Ragas and LangSmith

Evaluation is crucial for assessing the performance of a RAG pipeline. We will use Ragas and LangSmith for this purpose.

## Setting up LangSmith (Optional but recommended)
LangSmith helps trace, debug, and monitor RAG applications.

## Evaluating with Ragas

Context precision measures the signal-to-noise ratio of the retrieved context. This metric is computed using the question and the contexts.
Context recall measures if all the relevant information required to answer the question was retrieved. This metric is computed based on the ground_truth (this is the only metric in the framework that relies on human-annotated ground truth labels) and the contexts.
Faithfulness measures the factual accuracy of the generated answer. The number of correct statements from the given contexts is divided by the total number of statements in the generated answer. This metric uses the question, contextsand the answer.
Answer relevancy measures how relevant the generated answer is to the question. This metric is computed using the question and the answer. For example, the answer “France is in western Europe.” to the question “Where is France and what is it’s capital?” would achieve a low answer relevancy because it only answers half of the question.

In [ ]:
# Inst1all Ragas if not already installed
!pip install ragas

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)
from datasets import Dataset

# Set the OPENAI_API_KEY environment variable for Ragas
import os
os.environ["OPENAI_API_KEY"] = personal_api_key

# Convert the list of evaluation data into a Ragas Dataset
# We need to flatten the nested structure of the 'data' list
flattened_data = []
for document_data in data:
    for category, questions in document_data.items():
        for question_data in questions.values():
            flattened_data.extend(question_data)

# Prepare data for Ragas Dataset with required columns
ragas_data = []
for item in flattened_data:
    # Run the query through the RAG chain to get retrieved contexts and generated answer
    rag_response = qa_chain.invoke(item["question"])

    ragas_data.append({
        "question": item["question"],
        "ground_truth_answer": item["ground_truth_answer"], # Keep original for reference
        "retrieved_contexts": [doc.page_content for doc in rag_response["source_documents"]], # Extract page content
        "reference": item["ground_truth_answer"], # Use ground truth answer as reference
        "answer": rag_response["result"] # Generated answer from the RAG chain
    })


# Convert the prepared data into a Ragas Dataset
eval_dataset = Dataset.from_list(ragas_data)


result = evaluate(
    dataset = eval_dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

In [ ]:
14# Calculate and print average and variance for each evaluation metric
print("Evaluation Metrics Averages and Variances:")
for metric in ['context_precision', 'context_recall', 'faithfulness', 'answer_relevancy']:
    if metric in df.columns:
        avg = df[metric].mean()
        variance = df[metric].var()
        print(f"{metric.replace('_', ' ').title()}:")
        print(f"  Average: {avg:.4f}")
        print(f"  Variance: {variance:.4f}")
        print("-" * 20)

# Explanation of evaluation metrics
print("\nExplanation of Evaluation Metrics:")
print("- Context Precision: Measures the signal-to-noise ratio of the retrieved context. It assesses how relevant the retrieved information is to the question.")
print("- Context Recall: Measures if all the relevant information required to answer the question was retrieved. It is based on the human-annotated ground truth answer.")
print("- Faithfulness: Measures the factual accuracy of the generated answer based on the retrieved context. It checks if statements in the answer are supported by the context.")
print("- Answer Relevancy: Measures how relevant the generated answer is to the question. It assesses if the answer directly addresses the user's query.")
